<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 初期設定

In [ ]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/')

Mounted at /content/drive


In [ ]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=84838219585112425c46d272177dc16a538d1a6818f95b896c06e9d925e354b1
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
!pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=16c5ae2ffa4b02969652699b60fa01d10c5e3862fdd9d8e569e37e03dc5295bd
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [ ]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

In [ ]:
get_stock_data_yapi2(9501)

,Open,High,Low,Close,Volume
Date,,,,,
2013-06-05,623.0,624.0,513.0,513.0,233103300
2013-06-06,477.0,506.0,451.0,475.0,430786400
2013-06-07,499.0,534.0,485.0,515.0,324742800
2013-06-10,534.0,589.0,526.0,565.0,324329800
2013-06-11,555.0,560.0,523.0,534.0,169504300
...,...,...,...,...,...
2023-05-29,477.0,480.0,474.0,480.0,14262300
2023-05-30,481.0,487.0,478.0,487.0,13062100
2023-05-31,487.0,491.0,479.0,481.0,28232900


# JPX400の銘柄一覧を取得する関数

In [ ]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_jpx400():
    """JPX400の銘柄コード一覧を取得する

    :return: JPX400の銘柄コード一覧
    """

    url = "https://indexes.nikkei.co.jp/nkave/archives/file/jpx_nikkei_index_400_weight_jp.csv"
    r = requests.get(url)
    with open('jpx_nikkei_index_400_weight_jp.csv', 'wb') as output:
        output.write(r.content)

    df = pd.read_csv("./jpx_nikkei_index_400_weight_jp.csv", encoding='shift_jis')
    # 不要なレコードとカラムを削除
    df = df.iloc[:-1]   # 最後の行は説明文のため除外
    df = df[['銘柄名', 'コード', '業種']]   # 必要なカラムのみにする
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

In [ ]:
# JPX400の銘柄コードを取得
jpx400 = get_ticker_jpx400()
jpx400

,銘柄名,コード,業種
0,ニッスイ,1332,水産・農林業
1,ウエストホールディングス,1407,建設業
2,ショーボンドホールディングス,1414,建設業
3,ミライト・ワン,1417,建設業
4,ＩＮＰＥＸ,1605,鉱業
...,...,...,...
394,ニトリホールディングス,9843,小売業
395,ミスミグループ本社,9962,卸売業
396,ファーストリテイリング,9983,小売業
397,ソフトバンクグループ,9984,情報・通信業


In [ ]:
# ニッスイの株価情報を取得
get_stock_data_yapi2(jpx400.loc[jpx400['銘柄名'] == 'ニッスイ', 'コード'].values[0])

,Open,High,Low,Close,Volume
Date,,,,,
2013-06-04,197.0,226.0,197.0,212.0,17830600
2013-06-05,205.0,211.0,197.0,198.0,3604200
2013-06-06,198.0,200.0,191.0,192.0,3796300
2013-06-07,190.0,193.0,183.0,191.0,3263300
2013-06-10,193.0,199.0,192.0,198.0,1812000
...,...,...,...,...,...
2023-05-26,638.0,638.0,630.0,630.0,762900
2023-05-29,635.0,641.0,634.0,636.0,1008500
2023-05-30,635.0,637.0,631.0,634.0,706000


# 東証上場の銘柄一覧を取得する関数

In [ ]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    r = requests.get(url)
    with open('data_j.xls', 'wb') as output:
        output.write(r.content)

    df = pd.read_excel("./data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [ ]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [ ]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    # 空のDataFrameを作成
    result_df = pd.DataFrame()

    # 各銘柄についてループ
    for index, row in df.iterrows():
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        filtered_df = pd.DataFrame({
            '銘柄名': [row['銘柄名']],
            'コード': [code],
            '業種': [row['業種']],
            '出来高': [volume],
            '株価': [price],
        }, index=[index])

        # 結果をresult_dfに追加
        result_df = pd.concat([result_df, filtered_df])

    return result_df

# JPX400をフィルタリング


In [ ]:
# JPX400の銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 500円〜5000円
filtered_jpx400 = filtered_ticker(jpx400, min_volume=200000, max_price=5000, min_price=500)
filtered_jpx400

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,937000,622.0
1,ウエストホールディングス,1407,建設業,286900,2776.0
4,ＩＮＰＥＸ,1605,鉱業,5409700,1486.0
5,安藤・間,1719,建設業,499900,1024.0
6,コムシスホールディングス,1721,建設業,444300,2729.0
...,...,...,...,...,...
380,大阪瓦斯,9532,電気・ガス業,1133400,2207.0
382,エヌ・ティ・ティ・データ,9613,情報・通信業,2179100,2007.0
387,ＳＣＳＫ,9719,情報・通信業,776200,2253.0
395,ミスミグループ本社,9962,卸売業,806300,3085.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_jpx400 = filtered_ticker(filtered_jpx400, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_jpx400

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1230100,626.0
3,ミライト・ワン,1417,建設業,314700,1741.0
5,安藤・間,1719,建設業,1126100,1011.0
6,コムシスホールディングス,1721,建設業,674700,2705.0
7,大成建設,1801,建設業,2491600,4435.0
...,...,...,...,...,...
375,北海道電力,9509,電気・ガス業,2198900,572.0
379,東京瓦斯,9531,電気・ガス業,3559700,2969.0
390,ＮＳＤ,9759,情報・通信業,344200,2704.0
395,ミスミグループ本社,9962,卸売業,1883800,3050.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_jpx400 = filtered_ticker(filtered_perfect_order_week_jpx400,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_jpx400

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1230100,626.0
3,ミライト・ワン,1417,建設業,314700,1741.0
5,安藤・間,1719,建設業,1126100,1011.0
6,コムシスホールディングス,1721,建設業,674700,2705.0
8,大林組,1802,建設業,7666300,1124.0
...,...,...,...,...,...
373,関西電力,9503,電気・ガス業,3743200,1596.0
375,北海道電力,9509,電気・ガス業,2198900,572.0
379,東京瓦斯,9531,電気・ガス業,3559700,2969.0
390,ＮＳＤ,9759,情報・通信業,344200,2704.0


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_jpx400 = filtered_ticker(filtered_jpx400, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_jpx400

,銘柄名,コード,業種,出来高,株価
1,ウエストホールディングス,1407,建設業,619300,2837.000000
12,西松建設,1820,建設業,332100,3350.000000
31,明治ホールディングス,2269,食料品,2035900,3125.000000
34,システナ,2317,情報・通信業,1542000,303.000000
38,ディップ,2379,サービス業,276500,3370.000000
40,ベネフィット・ワン,2412,サービス業,827900,1533.000000
41,エムスリー,2413,サービス業,2587600,3108.000000
43,バリューコマース,2491,サービス業,222400,1403.000000
52,セリア,2782,小売業,277200,2267.000000
79,シップヘルスケアホールディングス,3360,卸売業,356600,2366.000000


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_jpx400 = filtered_ticker(filtered_perfect_order_week_jpx400,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_perfect_order_week_day_jpx400

,銘柄名,コード,業種,出来高,株価
170,コスモエネルギーホールディングス,5021,石油・石炭製品,845000,3915.0
177,太平洋セメント,5233,ガラス・土石製品,591900,2373.0
178,東海カーボン,5301,ガラス・土石製品,2506600,1156.0
224,ジャパンエレベーターサービスホールディングス,6544,サービス業,382800,1879.0


# 東証プライムをフィルタリング

In [ ]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4259,サンドラッグ,9989,小売業
4260,サックスバー ホールディングス,9990,小売業
4261,ジェコス,9991,卸売業
4264,グローセル,9995,卸売業


In [ ]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, max_price=5000)
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,1665900,645.599976
159,ＩＮＰＥＸ,1605,鉱業,7315700,1586.500000
223,安藤・間,1719,建設業,863200,1059.000000
225,コムシスホールディングス,1721,建設業,804800,2807.500000
245,オリエンタル白石,1786,建設業,1097000,295.000000
...,...,...,...,...,...
4086,エイチ・アイ・エス,9603,サービス業,782600,1900.000000
4090,エヌ・ティ・ティ・データ,9613,情報・通信業,4745600,2048.500000
4138,ＳＣＳＫ,9719,情報・通信業,660800,2263.500000
4188,ヤマダホールディングス,9831,小売業,4486200,429.000000


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,1665900,645.599976
223,安藤・間,1719,建設業,863200,1059.000000
225,コムシスホールディングス,1721,建設業,804800,2807.500000
253,大成建設,1801,建設業,967200,4579.000000
254,大林組,1802,建設業,3076800,1177.500000
...,...,...,...,...,...
4049,北陸電力,9505,電気・ガス業,2187000,803.700012
4050,東北電力,9506,電気・ガス業,5066100,887.500000
4051,四国電力,9507,電気・ガス業,1492200,1024.000000
4053,北海道電力,9509,電気・ガス業,1565300,616.599976


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,1665900,645.599976
223,安藤・間,1719,建設業,863200,1059.000000
225,コムシスホールディングス,1721,建設業,804800,2807.500000
254,大林組,1802,建設業,3076800,1177.500000
255,清水建設,1803,建設業,2942300,878.000000
...,...,...,...,...,...
4049,北陸電力,9505,電気・ガス業,2187000,803.700012
4050,東北電力,9506,電気・ガス業,5066100,887.500000
4051,四国電力,9507,電気・ガス業,1492200,1024.000000
4053,北海道電力,9509,電気・ガス業,1565300,616.599976


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
266,三井住友建設,1821,建設業,699100.0,365.000000
392,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,2678900.0,989.599976
463,明治ホールディングス,2269,食料品,1038300.0,3235.000000
483,システナ,2317,情報・通信業,1564700.0,300.000000
495,いちご,2337,不動産業,1161400.0,269.000000
530,エムスリー,2413,サービス業,2694100.0,3175.000000
556,エスプール,2471,サービス業,1505700.0,535.000000
569,インフォマート,2492,サービス業,2257900.0,271.000000
626,伊藤園,2593,食料品,685500.0,4258.000000
911,東洋紡,3101,繊維製品,536800.0,1000.500000


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
266,三井住友建設,1821,建設業,699100,365.000000
463,明治ホールディングス,2269,食料品,1038300,3235.000000
556,エスプール,2471,サービス業,1505700,535.000000
569,インフォマート,2492,サービス業,2257900,271.000000
911,東洋紡,3101,繊維製品,536800,1000.500000
1203,グリー,3632,情報・通信業,567700,673.000000
1422,住友化学,4005,化学,9287700,425.899994
1459,デンカ,4061,化学,708800,2634.000000
1730,住友ファーマ,4506,医薬品,1837200,650.200012
1868,サイバーエージェント,4751,サービス業,3124700,1029.000000


# 東証スタンダードをフィルタリング

In [ ]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4254,タキヒヨー,9982,卸売業
4257,蔵王産業,9986,卸売業
4262,ヤマザワ,9993,小売業
4263,やまや,9994,小売業


In [ ]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, max_price=5000)
filtered_tse_standard

,銘柄名,コード,業種,出来高,株価
57,ウエストホールディングス,1407,建設業,300900,2553.0
120,住石ホールディングス,1514,鉱業,463100,372.0
233,中小企業ホールディングス,1757,建設業,851000,47.0
244,アジアゲートホールディングス,1783,建設業,543700,52.0
275,大盛工業,1844,建設業,330800,195.0
...,...,...,...,...,...
4004,ビート・ホールディングス・リミテッド,9399,情報・通信業,1215451,7.0
4018,フォーバル・リアルストレート,9423,情報・通信業,270800,98.0
4088,ウィルソン・ラーニング ワールドワイド,9610,サービス業,2368800,151.0
4130,アゴーラ ホスピタリティー グループ,9704,サービス業,585200,26.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
233,中小企業ホールディングス,1757,建設業,851000,47.0
275,大盛工業,1844,建設業,330800,195.0
396,クルーズ,2138,情報・通信業,471200,906.0
540,シダー,2435,サービス業,217900,246.0
566,ＪＴＰ,2488,サービス業,385800,1050.0
995,コーセーアールイー,3246,不動産業,299300,788.0
1192,クラウディアホールディングス,3607,繊維製品,393400,653.0
1229,ｅｎｉｓｈ,3667,情報・通信業,5000900,848.0
1314,パス,3840,情報・通信業,1783700,116.0
1415,ＰＫＳＨＡ Ｔｅｃｈｎｏｌｏｇｙ,3993,情報・通信業,1152800,2758.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
233,中小企業ホールディングス,1757,建設業,851000,47.0
275,大盛工業,1844,建設業,330800,195.0
995,コーセーアールイー,3246,不動産業,299300,788.0
1192,クラウディアホールディングス,3607,繊維製品,393400,653.0
1229,ｅｎｉｓｈ,3667,情報・通信業,5000900,848.0
1314,パス,3840,情報・通信業,1783700,116.0
1415,ＰＫＳＨＡ Ｔｅｃｈｎｏｌｏｇｙ,3993,情報・通信業,1152800,2758.0
1602,ユークス,4334,情報・通信業,254400,1946.0
1848,ＫｅｙＨｏｌｄｅｒ,4712,サービス業,336300,1196.0
2249,ヤマシナ,5955,金属製品,2247200,102.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
57,ウエストホールディングス,1407,建設業,300900,2553.0
244,アジアゲートホールディングス,1783,建設業,543700,52.0
482,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,2582200,60.0
501,クシム,2345,情報・通信業,401500,386.0
563,出前館,2484,情報・通信業,1703100,438.0
673,イメージ ワン,2667,卸売業,513100,713.0
1301,サイバーステップ,3810,情報・通信業,247200,486.0
1307,リミックスポイント,3825,情報・通信業,735600,240.0
1720,コンピューターマネージメント,4491,情報・通信業,231900,1529.0
2014,昭和ホールディングス,5103,ゴム製品,272700,44.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
57,ウエストホールディングス,1407,建設業,300900,2553.0
244,アジアゲートホールディングス,1783,建設業,543700,52.0
1307,リミックスポイント,3825,情報・通信業,735600,240.0
2014,昭和ホールディングス,5103,ゴム製品,272700,44.0
2718,ウインテスト,6721,電気機器,386800,108.0
2724,ピクセラ,6731,電気機器,3991300,3.0
2867,大黒屋ホールディングス,6993,小売業,691100,53.0
3604,井筒屋,8260,小売業,859500,327.0
3771,ランド,8918,不動産業,24612100,9.0
3933,ビジョナリーホールディングス,9263,小売業,564900,112.0


# 東証グロースをフィルタリング

In [ ]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4077,プログリット,9560,サービス業
4078,グラッドキューブ,9561,サービス業
4079,ビジネスコーチ,9562,サービス業
4080,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [ ]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
400,ケアネット,2150,サービス業,387300,824.0
406,ＦＲＯＮＴＥＯ,2158,サービス業,617100,661.0
407,ジーエヌアイグループ,2160,医薬品,329900,1099.0
424,アミタホールディングス,2195,サービス業,272200,818.0
493,イオレ,2334,サービス業,289800,1728.0
...,...,...,...,...,...
3975,ＧＥＮＯＶＡ,9341,サービス業,354600,2186.0
3982,ｉｓｐａｃｅ,9348,サービス業,779200,1509.0
4059,リニューアブル・ジャパン,9522,電気・ガス業,845800,795.0
4071,マイクロアド,9553,サービス業,406000,4930.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
493,イオレ,2334,サービス業,289800,1728.0
559,ジェイテック,2479,サービス業,260400,317.0
926,海帆,3133,小売業,1938000,505.0
1281,アドバンスト・メディア,3773,情報・通信業,381200,1672.0
1351,Ａｉｍｉｎｇ,3911,情報・通信業,1397900,447.0
1411,シェアリングテクノロジー,3989,情報・通信業,690300,653.0
1450,フィーチャ,4052,情報・通信業,970500,1200.0
1453,ティアンドエス,4055,情報・通信業,571300,2671.0
1497,プレイド,4165,情報・通信業,724000,772.0
1501,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,397100,1379.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
559,ジェイテック,2479,サービス業,260400,317.0
1411,シェアリングテクノロジー,3989,情報・通信業,690300,653.0
1450,フィーチャ,4052,情報・通信業,970500,1200.0
1453,ティアンドエス,4055,情報・通信業,571300,2671.0
1497,プレイド,4165,情報・通信業,724000,772.0
1501,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,397100,1379.0
1568,セキュア,4264,情報・通信業,325200,2850.0
1664,ＪＤＳＣ,4418,情報・通信業,280300,1135.0
1665,Ｆｉｎａｔｅｘｔホールディングス,4419,情報・通信業,220800,809.0
1694,Ｃｈａｔｗｏｒｋ,4448,情報・通信業,405200,1191.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
400,ケアネット,2150,サービス業,387300,824.0
406,ＦＲＯＮＴＥＯ,2158,サービス業,617100,661.0
407,ジーエヌアイグループ,2160,医薬品,329900,1099.0
507,メディネット,2370,サービス業,549700,68.0
1198,アクセルマーク,3624,情報・通信業,281700,280.0
1287,ディー・ディー・エス,3782,情報・通信業,382800,53.0
1348,コラボス,3908,情報・通信業,957000,365.0
1427,カラダノート,4014,情報・通信業,1024500,869.0
1508,ココナラ,4176,情報・通信業,462100,366.0
1591,ビーマップ,4316,情報・通信業,446100,472.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
1783,ヘリオス,4593,医薬品,673400,294.0
1787,ソレイジア・ファーマ,4597,医薬品,782700,45.0
3105,アイドマ・ホールディングス,7373,サービス業,297500,2349.0


# 東証スタンダード・プライム選抜


In [ ]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, max_price=5000)
filtered_tse_standard_2

,銘柄名,コード,業種,出来高,株価
233,中小企業ホールディングス,1757,建設業,851000,47.0
244,アジアゲートホールディングス,1783,建設業,543700,52.0
394,燦キャピタルマネージメント,2134,サービス業,2626200,22.0
482,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,2582200,60.0
563,出前館,2484,情報・通信業,1703100,438.0
673,イメージ ワン,2667,卸売業,513100,713.0
1229,ｅｎｉｓｈ,3667,情報・通信業,5000900,848.0
1245,ディー・エル・イー,3686,情報・通信業,1178800,339.0
1267,インタートレード,3747,情報・通信業,5996600,540.0
1307,リミックスポイント,3825,情報・通信業,735600,240.0


In [ ]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_grows, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
406,ＦＲＯＮＴＥＯ,2158,サービス業,617100,661.0
507,メディネット,2370,サービス業,549700,68.0
518,ウェッジホールディングス,2388,その他金融業,4150600,104.0
926,海帆,3133,小売業,1938000,505.0
1226,モブキャストホールディングス,3664,情報・通信業,613000,83.0
1284,ＦＨＴホールディングス,3777,情報・通信業,772500,38.0
1348,コラボス,3908,情報・通信業,957000,365.0
1351,Ａｉｍｉｎｇ,3911,情報・通信業,1397900,447.0
1411,シェアリングテクノロジー,3989,情報・通信業,690300,653.0
1427,カラダノート,4014,情報・通信業,1024500,869.0
